In [ ]:
import numpy as np
import pandas as pd

from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from tqdm import tqdm
from sklearn.metrics import classification_report

- 모델 로드

In [ ]:
base_url = "" ## url과 api key는 보안 이슈로 지웠습니다!
api_key = ""
max_tokens = 2048
temperature = 0.1

client = OpenAI(base_url=base_url, api_key=api_key)
models = client.models.list()
model = models.data[0].id

llm = ChatOpenAI(
                model=model,
                openai_api_key=api_key,
                openai_api_base=base_url,
                max_tokens=max_tokens,
                temperature=temperature,
            )

## 광고성 리뷰 분류

- 광고성 리뷰 분류를 위한 시스템 프롬프트 정의 및 LLM 체인 생성

In [ ]:
system_prompt = """제품 구입 후기에 대한 리뷰 텍스트가 주어집니다. 당신은 리뷰 텍스트 중 광고성 리뷰 텍스트에 대한 이진 분류를 수행해야합니다.
광고성 리뷰는 소비자가 제품이나 서비스를 직접 경험한 결과를 토대로 작성된 솔직한 리뷰와는 달리, 특정 제품이나 서비스를 홍보하려는 의도를 가진 리뷰를 의미합니다.
광고성 리뷰의 특징은 아래와 같습니다.
1. 구체적인 사용 경험 부족
  - 제품 사용 경험에 대한 구체적인 정보(어떻게 사용했는지, 사용 환경, 상황 등)가 부족하거나 모호함
2. 경쟁사의 제품을 의도적으로 깎아내림
  - 특정 경쟁 제품을 부정적으로 비교하며 구매를 유도하려는 리뷰
3. 구매를 강력히 권장하는 어조
  - "이 제품을 사지 않으면 후회할 것이다", "친구들에게도 추천했다", "여러 개를 구매할 예정이다" 등 구매를 강요하거나 권장하는 내용
4. 구매 링크 또는 홍보성 멘트 포함
  - "이 링크를 통해 구매하세요", "공식 홈페이지에서 구매하면 더 좋다" 등 직접적인 구매 유도"""

prompt_template = ChatPromptTemplate.from_messages([("system", system_prompt), ("human", "###Context###:\n{context}\n\n###Question###:\n{question}\n\n###Answer###:")])
chain = prompt_template | llm | StrOutputParser()

- 리뷰 텍스트 전처리

In [ ]:
df = pd.read_csv('./sample_data/유닉스매직기_광고성리뷰_포함.csv')
plain_review_list = list(df[df['광고성리뷰'] == 0]['리뷰내용'])
adver_review_list = list(df[df['광고성리뷰'] == 1]['리뷰내용'])

review_list = plain_review_list + adver_review_list
gt_list = ['no' for i in range(len(plain_review_list))] + ['yes' for i in range(len(adver_review_list))]

In [ ]:
adcls_result_list = []
question_text = "광고성 리뷰의 특징을 참고하여 주어진 리뷰 텍스트가 광고성 리뷰이면 yes, 광고성 리뷰가 아니면 no로만 답하여 주세요."
for review in tqdm(review_list, total=len(review_list)):
    result = chain.invoke({"context":review, "question": question_text})
    adcls_result_list.append(result)

100%|██████████| 100/100 [00:14<00:00,  6.68it/s]


- 분류 결과 : 18개의 리뷰 데이터에 대한 광고성 리뷰 분류 정확도 88% 달성

In [ ]:
for review, cls_result, gt in zip(review_list, adcls_result_list, gt_list):
    print("리뷰 내용 : {}".format(review))
    print("광고성 리뷰 분류 결과 : {}    정답 :  {}".format(cls_result, gt))
    print("-------------------------------------------------------")

리뷰 내용 : 유닉스 세라믹 헤어 매직기 UCI-A2521 상세 리뷰

유닉스 세라믹 헤어 매직기 UCI-A2521은 가정용으로 사용하기에 적합한 합리적인 가격대의 헤어 스타일링 기기입니다. 헤어 관리에 관심이 많은 사람들에게 유용한 제품으로, 간단한 조작과 안정적인 성능이 특징입니다. 제가 직접 사용한 것은 아니지만, 아내가 며칠간 사용해본 후 느낀 점과 제 관찰을 바탕으로 리뷰를 작성해 봅니다.

우선 디자인은 심플하면서도 고급스러운 느낌을 줍니다. 그립감이 좋아 손이 작아도 안정적으로 잡을 수 있고, 가벼운 무게 덕분에 사용 중 팔에 무리가 덜합니다. 온도 조절 기능은 120°C에서 200°C까지 5단계로 나뉘어 있어, 모발 상태나 스타일링 목적에 따라 적절한 온도를 선택할 수 있습니다. 특히, 열판이 세라믹 코팅 처리되어 있어 열 전달이 고르게 이루어지고, 머리카락 손상을 최소화하려는 설계가 돋보입니다.

사용 후 효과

아내의 말을 빌리자면, “머리가 부드럽게 잘 펴지고, 스타일링 후에도 자연스러운 광택이 남아 기분이 좋았다”고 합니다. 실제로 사용 후 머리카락이 뻣뻣하거나 건조해 보이지 않았고, 부드러운 느낌이 유지되는 것을 확인할 수 있었습니다. 평소 곱슬기가 있는 편이라 매직기로 머리를 펴는 데 시간이 꽤 걸리곤 했는데, UCI-A2521은 열판이 넓고 균일한 열 전달 덕분에 스타일링 시간이 크게 단축됐습니다.

또한, 예열 시간이 짧아 바쁜 아침에도 빠르게 사용할 수 있는 점도 장점입니다. 30초 정도면 원하는 온도에 도달해 바로 사용할 수 있어 시간을 절약할 수 있었습니다.

장점

1. 세라믹 열판: 균일한 열 전달로 모발 손상을 최소화하면서도 스타일링이 깔끔하게 됩니다.
2. 온도 조절 기능: 다양한 모발 상태에 맞춰 적절한 온도를 선택할 수 있어 유용합니다.
3. 간편한 사용성: 초보자도 손쉽게 사용할 수 있는 직관적인 조작과 빠른 예열 시간이 인상적입니다.
4. 안전 설계: 일정 시간 사용하지 않으면 자동으로 전원이 꺼지는 기능이 있어

- 분류 결과 정확도
- f1-score : 88.99%, accuracy : 89.00%

In [ ]:
print(classification_report(gt_list, adcls_result_list, digits=4))

              precision    recall  f1-score   support

          no     0.9149    0.8600    0.8866        50
         yes     0.8679    0.9200    0.8932        50

    accuracy                         0.8900       100
   macro avg     0.8914    0.8900    0.8899       100
weighted avg     0.8914    0.8900    0.8899       100



- 광고성 리뷰가 아닌 텍스트 필터링

In [ ]:
target_review_idx = np.where(np.array(adcls_result_list) == 'no')[0]
target_review_list = [review_list[target_review_idx[i]] for i in range(len(target_review_idx))]
target_review = ""

for i, review in enumerate(target_review_list):
    target_review += "리뷰 {} : {}\n\n".format(str(i+1), review)

In [ ]:
target_review_list, len(target_review_list)

(['유닉스 세라믹 헤어 매직기 UCI-A2521 상세 리뷰\n\n유닉스 세라믹 헤어 매직기 UCI-A2521은 가정용으로 사용하기에 적합한 합리적인 가격대의 헤어 스타일링 기기입니다. 헤어 관리에 관심이 많은 사람들에게 유용한 제품으로, 간단한 조작과 안정적인 성능이 특징입니다. 제가 직접 사용한 것은 아니지만, 아내가 며칠간 사용해본 후 느낀 점과 제 관찰을 바탕으로 리뷰를 작성해 봅니다.\n\n우선 디자인은 심플하면서도 고급스러운 느낌을 줍니다. 그립감이 좋아 손이 작아도 안정적으로 잡을 수 있고, 가벼운 무게 덕분에 사용 중 팔에 무리가 덜합니다. 온도 조절 기능은 120°C에서 200°C까지 5단계로 나뉘어 있어, 모발 상태나 스타일링 목적에 따라 적절한 온도를 선택할 수 있습니다. 특히, 열판이 세라믹 코팅 처리되어 있어 열 전달이 고르게 이루어지고, 머리카락 손상을 최소화하려는 설계가 돋보입니다.\n\n사용 후 효과\n\n아내의 말을 빌리자면, “머리가 부드럽게 잘 펴지고, 스타일링 후에도 자연스러운 광택이 남아 기분이 좋았다”고 합니다. 실제로 사용 후 머리카락이 뻣뻣하거나 건조해 보이지 않았고, 부드러운 느낌이 유지되는 것을 확인할 수 있었습니다. 평소 곱슬기가 있는 편이라 매직기로 머리를 펴는 데 시간이 꽤 걸리곤 했는데, UCI-A2521은 열판이 넓고 균일한 열 전달 덕분에 스타일링 시간이 크게 단축됐습니다.\n\n또한, 예열 시간이 짧아 바쁜 아침에도 빠르게 사용할 수 있는 점도 장점입니다. 30초 정도면 원하는 온도에 도달해 바로 사용할 수 있어 시간을 절약할 수 있었습니다.\n\n장점\n\n1. 세라믹 열판: 균일한 열 전달로 모발 손상을 최소화하면서도 스타일링이 깔끔하게 됩니다.\n2. 온도 조절 기능: 다양한 모발 상태에 맞춰 적절한 온도를 선택할 수 있어 유용합니다.\n3. 간편한 사용성: 초보자도 손쉽게 사용할 수 있는 직관적인 조작과 빠른 예열 시간이 인상적입니다.\n4. 안전 설계: 일정 시간 사용하지 않으면 자동으로 전원

## 리뷰 텍스트 요약

- 리뷰 요약을 위한 시스템 프롬프트 및 체인 생성

In [ ]:
system_prompt = """제품 구입 후기에 대한 다수의 리뷰 텍스트가 주어집니다.
당신은 다수의 리뷰 텍스트에서 제품에 대한 구체적인 설명, 사용 경험 공유 내용, 장점과 단점에 대한 요약 텍스트를 생성해주세요
답변을 작성할 때 아래와 같은 규칙을 지켜주세요.
1. 답변은 분석적이어야 하며 전문적인 어조를 사용해야 합니다.
2. 답변은 질문에 대한 자세한 설명을 제공해야 합니다.
3. 답변은 bullet points를 사용하여 독자가 이해하기 쉬운 계층 구조를 사용해야 합니다.
4. 답변은 Markdown format으로 작성되어야 합니다. 이것은 markdown coding이 아닙니다. ```markdown ```을 사용하지 마세요.
5. 답변에는 특정 기업 또는 기관에 대한 이름을 정확하게 작성해야하며 특정 기업 또는 기관의 이름을 생략하면 안됩니다."""

prompt_template = ChatPromptTemplate.from_messages([("system", system_prompt), ("human", "###Context###:\n{context}\n\n###Question###:\n{question}\n\n###Answer###:")])
chain = prompt_template | llm | StrOutputParser()

- 리뷰 요약 실행

In [ ]:
question_text = "다수의 리뷰 텍스트에서 제품에 대한 구체적인 설명, 사용 경험 공유 내용, 장점과 단점에 대한 요약 텍스트를 생성해주세요"

## 리뷰의 개수가 많아 10개만 언더 샘플링하여 수행, 운영 시 batched inference로 구현해야 할 듯...
sample_idx = np.random.choice(len(target_review_list), 10, replace=False)
sample_review_list = [target_review_list[sample_idx[i]] for i in range(len(sample_idx))]

result = chain.invoke({"context":sample_review_list, "question": question_text})

In [ ]:
print(result)

**제품 설명 및 기능**
- **제품 모델**: 유닉스 세라믹 헤어 고데기 UCI-A2521
- **주요 기능**
  - 세라믹 코팅: 모발 보호를 위한 원적외선 발생
  - 온도 조절: 120도에서 200도까지 5단계 조절 가능
  - 슬라이드 방식의 온오프 스위치
  - 360도 회전식 코드: 머리 꼬임 방지
  - LED 온도 표시: 설정 온도 도달 시 녹색으로 변하는 기능

**사용 경험 및 만족도**
- **사용 목적**: 곱슬머리 사용자에게 적합한 제품으로, 빠른 열전달과 모발 보호를 목표로 함
- **사용자 경험**
  - 사용 후 머리카락이 부드럽고 자연스러운 스타일 유지
  - 빠른 예열 시간: 30초 이내로 원하는 온도 도달
  - 사용 편의성: 초보자도 쉽게 사용 가능
  - 안전성: 일정 시간 사용하지 않으면 자동으로 전원 꺼짐

**장점**
- **세라믹 코팅**: 모발 손상 최소화
- **온도 조절**: 다양한 모발 상태에 맞춰 온도 조절 가능
- **사용성**: 직관적인 조작과 빠른 예열 시간
- **안전성**: 자동 전원 끄기 기능

**단점 및 개선점**
- **전선 길이**: 콘센트가 멀리 있는 곳에서 사용 시 불편
- **외부 열판 온도**: 사용 시 외부 열판이 뜨겁게 느껴질 수 있음

**결론**
- 유닉스 세라믹 헤어 고데기 UCI-A2521은 합리적인 가격대에 안정적인 성능을 제공하며, 특히 곱슬기나 부스스한 모발을 가진 사용자에게 적합
- 제품의 다양한 장점과 사용 편의성으로 인해 만족스러운 선택이 될 수 있음
